In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']
print(openai.api_key )

### Caching: LangChain为llm提供了一个可选的缓存层。这有两个有用的原因:
如果您经常多次请求相同的完成，它可以通过减少对LLM提供程序的API调用数量来为您节省资金。它可以通过减少对LLM提供程序的API调用数 量来加快应用程序的速度。

In [ ]:
import langchain
from langchain.llms import OpenAI

In [ ]:
# To make the caching really obvious, lets use a slower model.
llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)

In [ ]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

In [ ]:
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

### SQLite Cache

In [ ]:
# We can do the same thing with a SQLite cache
from langchain.cache import SQLiteCache
langchain.llm_cache = SQLiteCache(database_path=".langchain.db")

In [ ]:
# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a joke")

In [ ]:
# The second time it is, so it goes faster
llm.predict("Tell me a joke")

### 也可以有选择的开关Caching

In [ ]:
# cache=False 先关闭：
llm = OpenAI(model_name="text-davinci-002")
no_cache_llm = OpenAI(model_name="text-davinci-002", cache=False)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain

text_splitter = CharacterTextSplitter()

In [14]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

file = os.environ['FILE_PATH']
print(file)

../data/


In [15]:
with open(file+"sample.txt") as f:
    state_of_the_union = f.read()
texts = text_splitter.split_text(state_of_the_union)

In [16]:
from langchain.docstore.document import Document
docs = [Document(page_content=t) for t in texts[:3]]
from langchain.chains.summarize import load_summarize_chain

In [17]:
chain = load_summarize_chain(llm, chain_type="map_reduce", reduce_llm=no_cache_llm)
chain.run(docs)

'\n\nThe Sierra Nevada mountains have a great deal of variation in snowfall from year to year, with some years seeing over 40 feet of snow. This year has been particularly snowy, with some areas receiving over 20 feet of snow. The snow is gradually melting as summer approaches, but some of it will remain in the form of glaciers. Lake Tahoe is the largest and most beautiful of the glacier lakes in California.'

In [18]:
chain.run(docs)

'\n\nThe Sierra Nevada mountains have a great deal of variation in snowfall from year to year, with some seasons seeing over 47 feet of snow. The lakes in the Sierra Nevada range were formed at the end of the ice period and are now popular tourist destinations.'